## Assistant API with Multi-turn Conversation

Once your create a Assistant, you can call the api via the Assistant.ip and Thread.ip 
Each thread id is a conversation. For a given ASSISTANT ID and THREAD ID, you can create a message, that message will automaticlly added into the chat history. That is the difference between ASSISTANT and Chat Completions

In [63]:
import os
from openai import OpenAI
import gradio as gr

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

/Users/tianyuliu/Library/Caches/pypoetry/virtualenvs/nlp-examples-2S-piTS9-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Check the Assistant
$\textcolor{green}{Get\ the\ personal\ Assistant\ API}$

In [53]:
import pandas as pd
# List all assistant
my_assistants = client.beta.assistants.list(
    order="desc", limit="20",
)
personal_asistant_id = my_assistants.data[0].id
print(my_assistants.data[0].id)
# my_assistants.data
pd.DataFrame(my_assistants.data)

# Delete all assistant
# response = client.beta.assistants.delete(assistants_id)

asst_neKIxzX9azeMqYlnYYT700AT


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"(id, asst_neKIxzX9azeMqYlnYYT700AT)","(created_at, 1729169843)","(description, None)","(instructions, You are an personal expert. Use...","(metadata, {})","(model, gpt-4o)","(name, Personal assistant)","(object, assistant)","(tools, [FileSearchTool(type='file_search', fi...","(response_format, auto)","(temperature, 1.0)","(tool_resources, ToolResources(code_interprete...","(top_p, 1.0)"


In [54]:
thread = client.beta.threads.create()

### Thread (Single Round Conversation)
Create a MESSAGE to support the single round conversation with annotation

In [59]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="How many industrial experience does Tianyu Liu have?"
)

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=personal_asistant_id,
)


messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Tianyu Liu has a total of 9 months of industrial experience based on his roles:

1. Worked as a Data Scientist at Munich Reinsurance Australia for 6 months (August 2023 - February 2024)[0].
2. Worked as an Applied Scientist at NinjaTech AI for 3 months (March 2024 - June 2024)[0].
[0] Resume_applied_scientist.pdf
[1] Resume_applied_scientist.pdf


In [61]:
# Check all the message in the Thread vai Thread ID
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[Message(id='msg_LUTlXQnT91FwFHrD2R97kiE7', assistant_id='asst_neKIxzX9azeMqYlnYYT700AT', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=220, file_citation=FileCitation(file_id='file-j7ZYjgIW5kjYEy3iklxjMfGT'), start_index=185, text='【12:0†Resume_applied_scientist.pdf】', type='file_citation'), FileCitationAnnotation(end_index=344, file_citation=FileCitation(file_id='file-j7ZYjgIW5kjYEy3iklxjMfGT'), start_index=309, text='【12:0†Resume_applied_scientist.pdf】', type='file_citation')], value='Tianyu Liu has a total of 9 months of industrial experience based on his roles:\n\n1. Worked as a Data Scientist at Munich Reinsurance Australia for 6 months (August 2023 - February 2024)【12:0†Resume_applied_scientist.pdf】.\n2. Worked as an Applied Scientist at NinjaTech AI for 3 months (March 2024 - June 2024)【12:0†Resume_applied_scientist.pdf】.'), type='text')], created_at=1729182898, incomplete_at=None, incomplete_details=None, m

## Personal Assistant with Multi-turn conversation

In [69]:

# Function to create a new thread
def create_thread(message_content):
    """
    Creates a new thread and sends the first message.
    """
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": message_content
            }
        ]
    )
    print(f"Thread created with ID: {thread.id}")
    return thread.id

# Function to run the assistant within the thread
def run_assistant(assistant_id, thread_id):
    """
    Executes the assistant run in the created thread and returns the response.
    """
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    
    messages = list(client.beta.threads.messages.list(thread_id=thread_id, run_id=run.id))
    
    message_content = messages[0].content[0].text.value
    return message_content

# Function to send a message in the thread
def send_message(thread_id, message_content):
    """
    Sends a message to the assistant in the ongoing thread.
    """
    client.beta.threads.messages.create(
        thread_id=thread_id,
        content=message_content,
        role="user"
    )
    print(f"Message sent: {message_content}")

# Main loop for conversation
def multi_turn_conversation(assistant_id):
    """
    Handles multi-turn conversation with the assistant.
    """
    # Ask user for the initial message and create a thread
    user_input = input("You: ")
    thread_id = create_thread(user_input)

    while True:
        # Get the assistant's response
        assistant_response = run_assistant(assistant_id, thread_id)
        print(f"Assistant: {assistant_response}")
        
        # Get next user input
        user_input = input("You: ")
        
        if user_input.lower() in ['exit', 'quit']:
            print("Ending conversation.")
            break
        
        # Send the new user message and continue the conversation
        send_message(thread_id, user_input)

# Set your assistant ID (replace with actual assistant ID)
assistant_id = personal_asistant_id

# Start the multi-turn conversation
multi_turn_conversation(assistant_id)


Thread created with ID: thread_mBjd9rYGlKUQzajwrR0HzMrO
Assistant: Tianyu Liu has a total of 8 months of industrial experience. This includes 3 months as an Applied Scientist at NinjaTech AI and 6 months as a Data Scientist at Munich Reinsurance Australia【4:0†Resume_applied_scientist.pdf】.
Message sent: What is my last question
Assistant: Your last question was: "How many industrial experience does Tianyu Liu have?"
Ending conversation.


## Personal Assistant with Multi-turn conversation via Gradio WebUI

In [74]:
import gradio as gr

# Function to create a new thread
def create_thread(message_content):
    """
    Creates a new thread and sends the first message.
    """
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": message_content
            }
        ]
    )
    return thread.id

# Function to run the assistant within the thread
def run_assistant(assistant_id, thread_id):
    """
    Executes the assistant run in the created thread and returns the response.
    """
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    
    messages = list(client.beta.threads.messages.list(thread_id=thread_id, run_id=run.id))
    
    message_content = messages[0].content[0].text.value
    return message_content

# Function to send a message in the thread
def send_message(thread_id, message_content):
    """
    Sends a message to the assistant in the ongoing thread.
    """
    client.beta.threads.messages.create(
        thread_id=thread_id,
        content=message_content,
        role="user"
    )

# Function to handle conversation logic for Gradio, including chat history
def chat(assistant_id, thread_id, user_message, chat_history):
    """
    Sends a user message and returns the assistant's response and updated chat history.
    """
    if not thread_id:  # Check if thread_id is None or empty
        # Create a new thread if it's the first message
        thread_id = create_thread(user_message)
    else:
        # Send message in an existing thread
        send_message(thread_id, user_message)
    
    # Get assistant's response
    assistant_response = run_assistant(assistant_id, thread_id)
    
    # Update the chat history
    chat_history.append(("You", user_message))
    chat_history.append(("Assistant", assistant_response))
    
    return chat_history, thread_id

# Initial variables
assistant_id = personal_asistant_id
thread_id = None
chat_history = []

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# AI Personal Assistant")

    assistant_id_input = gr.Textbox(value=assistant_id, label="Assistant ID", visible=False)  # Set the assistant ID
    thread_id_input = gr.Textbox(value=None, label="Thread ID (for multi-turn)", visible=False)  # Hidden Thread ID
    
    chatbot = gr.Chatbot(label="Chat History")  # Chatbot component to show the conversation
    user_input = gr.Textbox(label="You", placeholder="Type your message here...")

    def process_chat(user_message, assistant_id_input, thread_id_input, chat_history):
        # If thread_id_input is empty or None, set it to None (to avoid passing empty strings)
        thread_id_input = thread_id_input if thread_id_input else None
        chat_history, new_thread_id = chat(assistant_id_input, thread_id_input, user_message, chat_history)
        return chat_history, new_thread_id
    
    user_input.submit(
        process_chat, 
        inputs=[user_input, assistant_id_input, thread_id_input, chatbot], 
        outputs=[chatbot, thread_id_input]
    )

# Launch Gradio interface
demo.launch()


/Users/tianyuliu/Library/Caches/pypoetry/virtualenvs/nlp-examples-2S-piTS9-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
